In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)


In [29]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
#실수 값으로 변환
x_train = torch.FloatTensor(x_train)
#정수 값으로 형 고정
y_train = torch.LongTensor(y_train)


In [32]:
y_one_hot = torch.zeros(8, 3)
print(y_train.unsqueeze(1))
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot)


tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [33]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)


공식 기준으로 코드 전개

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


y = Ax+b 기준으로 코드 전개

In [35]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    #matmul은 행렬의 곱을 의미합니다
    z = x_train.matmul(W) + b
    #cross_entropy는 log_softmax를 적용한 뒤, negative log likelihood를 계산합니다.
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.247725
Epoch  100/1000 Cost: 0.232558
Epoch  200/1000 Cost: 0.221291
Epoch  300/1000 Cost: 0.211031
Epoch  400/1000 Cost: 0.201647
Epoch  500/1000 Cost: 0.193031
Epoch  600/1000 Cost: 0.185093
Epoch  700/1000 Cost: 0.177759
Epoch  800/1000 Cost: 0.170962
Epoch  900/1000 Cost: 0.164646
Epoch 1000/1000 Cost: 0.158764


nn.Module로 구현하기

In [37]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)


# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 회귀 모델 클래스 정의
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RegressionModel, self).__init__()
        # 단순 선형 계층 (y = Ax + b)
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        # 회귀 문제에서는 보통 마지막에 활성화 함수를 사용하지 않음
        return self.linear(x)

# 예시: 입력 차원 4, 출력 차원 1인 회귀 모델 생성
model = RegressionModel(input_dim=4, output_dim=1)

# 옵티마이저 설정 (모델의 모든 파라미터를 대상으로)
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 손실 함수로 평균 제곱 오차(MSE) 사용
criterion = nn.MSELoss()

# 임의의 데이터 예시 (x_train과 y_train은 여러분의 데이터로 대체)
# 예시 데이터: x_train은 (N, 4) 크기, y_train은 (N, 1) 크기
x_train = torch.FloatTensor([
    [1, 2, 1, 1],
    [2, 1, 3, 2],
    [3, 1, 3, 4],
    [4, 1, 5, 5],
    [1, 7, 5, 5],
    [1, 2, 5, 6],
    [1, 6, 6, 6],
    [1, 7, 7, 7]
])
y_train = torch.FloatTensor([[2], [2.5], [3], [3.5], [1.5], [1.8], [1], [0.5]])

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 모델을 통한 예측
    prediction = model(x_train)  # shape: (8, 1)
    
    # 비용 함수 계산 (MSE)
    cost = criterion(prediction, y_train)
    
    # 기울기 초기화, 역전파, 파라미터 업데이트
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100 에폭마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))
